In [103]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, KFold 

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [65]:
cancer = load_breast_cancer()

X = cancer.data   #нампай массив
y = cancer.target

In [102]:
len(y),np.sum(y==0)

(569, 212)

__Видим что классы несбалансированны (всего 569 наблюдений, из которых 212 нулевого класса), поэтому в качестве метрики вместо аккураси будем использовать f1-меру__

In [66]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, random_state=1)

In [104]:
logreg = LogisticRegression(random_state=1).fit(X_train,y_train)

pred_train, pred_test = logreg.predict(X_train), logreg.predict(X_test)

print(f'Train: {f1_score(y_train, pred_train)} , Test: {f1_score(y_test, pred_test)}')

Train: 0.9560439560439561 , Test: 0.9545454545454546


__Теперь посчитаем все то же самое, но на масштабированных данных:__

In [105]:
scaler3 = MinMaxScaler().fit(X)
X_scaled = scaler3.transform(X)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X_scaled, y, random_state=1)

logreg3 = LogisticRegression(random_state=1).fit(X_train3,y_train3)

pred_train3, pred_test3 = logreg3.predict(X_train3), logreg3.predict(X_test3)

print(f'Train: {f1_score(y_train3, pred_train3)} , Test: {f1_score(y_test3, pred_test3)}')

Train: 0.9727767695099819 , Test: 0.967032967032967


__Делаем пайплайн__

In [106]:
pipe = make_pipeline(MinMaxScaler(), LogisticRegression(random_state=1))
pipe.fit(X_train, y_train)

pipe_pred_test = pipe.predict(X_test)
print(f'Test: {f1_score(y_test, pipe_pred_test)}')

Test: 0.967032967032967


__Получили то же самое что и до этого на масш. данныx - модель работает корректно__

Идем дальше

In [70]:
pipe.steps

[('minmaxscaler', MinMaxScaler()),
 ('logisticregression', LogisticRegression(random_state=1))]

In [71]:
params_grid = {'logisticregression__penalty': ['l1', 'l2'],  # не распознает пеналти почему-то
          'logisticregression__C': [0.01, 0.1, 1, 10, 100],
          'logisticregression__max_iter': [50, 100, 200, 300, 500],
          'logisticregression__solver': ['lbfgs', 'liblinear']} # 'liblinear' проверю т.к. он для малых датасетов
                                                                # тоже игнорирует параметр солвер
grid = GridSearchCV(pipe, params_grid, cv=5)
grid.fit(X_train, y_train)

# Смотрим лучшие параметры
grid.best_estimator_

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('logisticregression',
                 LogisticRegression(C=10, max_iter=50, random_state=1))])

Добираемся до коэффициентов модели:

In [72]:
grid.best_estimator_.named_steps["logisticregression"].coef_

array([[-2.74420262, -2.50667374, -2.64958624, -2.45421695, -0.66965232,
         0.93489905, -2.73551588, -3.47031245, -0.67609417,  2.27956463,
        -2.96081898, -0.27998362, -1.92631838, -1.80243336, -0.95225867,
         2.69219356,  0.28457239, -0.82829096,  0.2577377 ,  1.67881171,
        -4.94530719, -4.31776017, -4.2700511 , -3.65296409, -2.67972148,
        -0.06184301, -2.73936938, -4.8049408 , -3.20918207, -0.99899885]])

Полные  параметры модели:

In [73]:
grid.best_estimator_.named_steps["logisticregression"].get_params(deep=True)

{'C': 10,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 50,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 1,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

__Можем записать лучшую модель в переменную таким образом, но лучше тем что закоммичен, т.к. выявляем лучшую мы 1 раз, а далее при запуске кода заново нам не нужно чтобы поиск лучшей модели запускался заново__

In [107]:
best_pipe = grid.best_estimator_
# или make_pipeline(MinMaxScaler(), LogisticRegression(C=10, max_iter=50, random_state=1))

best_pipe.fit(X_train, y_train)

best_pipe_pred_test = best_pipe.predict(X_test)
print(f'Test: {f1_score(y_test, best_pipe_pred_test)}')

Test: 0.9720670391061453


Зададим несколько моделей МО в поиск и посмотрим какая лучше:

In [75]:
# Помним что только простые? линейные модели чувствительны к полиномам

pipe = Pipeline([('preprocessing', MinMaxScaler()),('polyfeat', PolynomialFeatures()), ('classifier', SVC())])

params_grid = [{'classifier': [SVC()], 'preprocessing': [MinMaxScaler()],'polyfeat': [PolynomialFeatures()],
                'classifier__random_state': [1],
                'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                'polyfeat__degree': [None]}, # выключил т.к. SVC сложная модель самам по себе
               
               {'classifier': [RandomForestClassifier()],'preprocessing': [None],'polyfeat': [None],
                'classifier__random_state': [1],
                'classifier__max_features': [2, 3, 5, 7, 10],
               'classifier__n_estimators': [50, 100, 150, 250, 400],
               'classifier__min_samples_leaf': [2, 5, 8, 12]},
               
              {'classifier': [LogisticRegression()],'preprocessing': [MinMaxScaler()],'polyfeat': [PolynomialFeatures()],
               'classifier__random_state': [1],
               'classifier__penalty': ['l1', 'l2'],  # не распознает пеналти кажется l2 по умолчанию
               'classifier__C': [0.01, 0.1, 1, 10, 100],
               'classifier__max_iter': [50, 100, 200, 300, 500],
               'classifier__solver': ['lbfgs', 'liblinear'],
               'polyfeat__degree': [1,2,3]}]

               
grid_check = GridSearchCV(pipe, params_grid, cv=5)
grid_check.fit(X_train, y_train)
grid_check.best_estimator_


Pipeline(steps=[('preprocessing', MinMaxScaler()),
                ('polyfeat', PolynomialFeatures(degree=1)),
                ('classifier',
                 LogisticRegression(C=10, max_iter=50, random_state=1))])

Итак, Логистическая регрессия лучше всего справляется с предсказаниями.

In [98]:
f1_score(y_train, grid_check.best_estimator_.predict(X_train)),\
f1_score(y_test, grid_check.best_estimator_.predict(X_test))

(0.9907578558225507, 0.9720670391061453)

__Посмотрим какую степень полиномов использует лучшая модель:__

In [76]:
grid_check.best_estimator_.named_steps["polyfeat"].get_params(deep=True)

{'degree': 1, 'include_bias': True, 'interaction_only': False, 'order': 'C'}

__Посмотрим все параметры LogisticRegression лучшей модели:__

In [77]:
grid_check.best_estimator_.named_steps["classifier"].get_params(deep=True)

{'C': 10,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 50,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 1,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

__Захотелось убедиться что SVC действительно проигрывает лог. регрессии:__

Насколько понял полиномы эффективны только в просты линейных моделях, SVC же сложная модель, но на всякий попробуем с полиномами

In [79]:
pipe2 = make_pipeline(MinMaxScaler(), PolynomialFeatures(),  SVC())

params_grid2 = {'polynomialfeatures__degree': [1,2,3],
                'svc__random_state': [1],
                'svc__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                'svc__C': [0.001, 0.01, 0.1, 1, 10, 100]}

grid_check2 = GridSearchCV(pipe2, params_grid2, cv=5, n_jobs=-1)
grid_check2.fit(X_train, y_train)

grid_check2.best_estimator_

(Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('polynomialfeatures', PolynomialFeatures()),
                 ('svc', SVC(C=10, gamma=0.1, random_state=1))]),
 0.958041958041958)

In [109]:
f1_score(y_test, grid_check2.best_estimator_.predict(X_test))

0.9662921348314608

__Действительно уступает, лучшая модель использует при этом вторую степень полиномов:__

In [95]:
grid_check2.best_estimator_.named_steps["polynomialfeatures"].get_params(deep=True), \
'*'*76, \
grid_check2.best_estimator_.named_steps["svc"].get_params(deep=True)

({'degree': 2, 'include_bias': True, 'interaction_only': False, 'order': 'C'},
 '****************************************************************************',
 {'C': 10,
  'break_ties': False,
  'cache_size': 200,
  'class_weight': None,
  'coef0': 0.0,
  'decision_function_shape': 'ovr',
  'degree': 3,
  'gamma': 0.1,
  'kernel': 'rbf',
  'max_iter': -1,
  'probability': False,
  'random_state': 1,
  'shrinking': True,
  'tol': 0.001,
  'verbose': False})

Ниже тестирую ручную лог рег. с гр.спуском, т.к. раньше выдавала лучше результат

In [117]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

def calc_logloss(y, y_pred):
    y_pred_min = 1e-4
    y_pred_max = 1 - 1e-4
    y_pred = np.clip(y_pred, y_pred_min, y_pred_max)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

# функция лог регрессии
def calc_pred_proba(X, W):
    z = np.dot(X, W)
    return 1/(1 + np.exp(-z))

# град спуск для лог регрессии
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.normal(size=(X.shape[1])) #np.random.randn(X.shape[1]) #np.zeros(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:   # выключил т.к. смысла нет - для себя делал чтоб понимать что работает
#            print(i, W, err)
    return W

# используем чтобы перевести вероятности в класс. Нужно правда найти правильный порог еще
def calc_pred(P):
    return np.where(P > 0.5, 1, 0)


W = eval_model(X_train3, y_train3, iterations=500, eta=1e-4)
to_pred_train = calc_pred_proba(X_train3, W)
hand_pred_train = calc_pred(to_pred_train)

to_pred_test = calc_pred_proba(X_test3, W)
hand_pred_test = calc_pred(to_pred_test)

print(f'Train: {f1_score(y_train3, hand_pred_train)} , Test: {f1_score(y_test3, hand_pred_test)}')

Train: 0.19402985074626866 , Test: 0.23703703703703705


In [116]:
checking  = pd.DataFrame()
checking['y_test'] = y_test3
checking['chance_test'] = to_pred_test
checking['hand_pred_test'] = hand_pred_test
checking

,y_test,chance_test,hand_pred_test
0,1,0.374637,1
1,0,0.367355,1
2,1,0.325055,1
3,0,0.716000,1
4,0,0.359988,1
...,...,...,...
138,1,0.328255,1
139,1,0.443535,1
140,0,0.553833,1
141,0,0.585881,1
